In [ ]:
# %% [markdown]
# # Masterarbeit: F1 - Analyse der Kompensationsforderungen
#
# Dieses Notebook dient der Analyse der Umfragedaten zur Beantwortung der ersten Forschungsfrage:
# _"Welche monetären Kompensationsforderungen stellen Haushalte in der Schweiz für die Teilnahme an einem Peak-Shaving-Programm unter Verwendung von Smart-Home-Technologien?"_
#
# **Struktur:**
# 1. Setup und Laden der Kerndaten (Q9 & Q10 Flexibilitätsdaten)
# 2. Laden und Integration weiterer relevanter Umfragedaten (Soziodemografie, Einstellungen)
# 3. Analyse der generellen Teilnahmebereitschaft (Q10 `incentive_choice`)
# 4. Detaillierte Analyse der monetären Kompensationsforderungen (Q10 `incentive_pct_required`)
# 5. Einfluss ausgewählter soziodemografischer Faktoren
# 6. (Optional) Einfluss von Geräte-Wichtigkeit (Q8) und DR-Affinität (Q11, Q12)
# 7. Zusammenfassung und Export von Ergebnissen

# %% [code]
# === Abschnitt 1: Setup und Laden der Kerndaten ===
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
# import matplotlib.pyplot as plt # Alternative für manche Plots
# import seaborn as sns          # Alternative für manche Plots
import os
import sys
from pathlib import Path

# --- Pfad-Setup ---
# Annahme: Das Notebook liegt in PowerE/scripts/F1_Analyse_Kompensationsforderungen/
try:
    NOTEBOOK_FILE_PATH = Path(__file__).resolve() # Funktioniert, wenn als Skript ausgeführt
    SCRIPT_DIR_F1 = NOTEBOOK_FILE_PATH.parent
except NameError: # Tritt auf, wenn __file__ nicht definiert ist (typisch für interaktive Notebook-Ausführung)
    SCRIPT_DIR_F1 = Path(os.getcwd()).resolve() # Nimmt das aktuelle Arbeitsverzeichnis
    print(f"Hinweis: __file__ nicht verfügbar, SCRIPT_DIR_F1 auf CWD gesetzt: {SCRIPT_DIR_F1}")

# Erwarte, dass SCRIPT_DIR_F1 jetzt PowerE/scripts/F1_Analyse_Kompensationsforderungen/ ist
if SCRIPT_DIR_F1.name == "F1_Analyse_Kompensationsforderungen" and SCRIPT_DIR_F1.parent.name == "scripts":
    PROJECT_ROOT_NB = SCRIPT_DIR_F1.parent.parent
else:
    # Fallback, falls die Ordnerstruktur anders ist oder das Notebook woanders gestartet wird
    PROJECT_ROOT_NB = Path(os.getcwd()).resolve() # Nimm CWD als Projekt-Root
    if PROJECT_ROOT_NB.name == "F1_Analyse_Kompensationsforderungen": PROJECT_ROOT_NB = PROJECT_ROOT_NB.parent.parent
    elif PROJECT_ROOT_NB.name == "scripts": PROJECT_ROOT_NB = PROJECT_ROOT_NB.parent
    print(f"WARNUNG: Projekt-Root-Bestimmung ist möglicherweise ungenau. PROJECT_ROOT_NB gesetzt auf: {PROJECT_ROOT_NB}")
    print("Stelle sicher, dass dies dein 'PowerE'-Verzeichnis ist.")


if str(PROJECT_ROOT_NB) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT_NB))
    print(f"Added project root '{PROJECT_ROOT_NB}' to sys.path")

print(f"Verwendeter Projekt-Root für dieses Notebook: {PROJECT_ROOT_NB}")

# --- Importe deiner Module ---
try:
    from src.logic.respondent_level_model.data_transformer import create_respondent_flexibility_df
    from src.data_loader.survey_loader.demographics import load_demographics
    from src.data_loader.survey_loader.attitudes import load_attitudes
    from src.data_loader.survey_loader.demand_response import load_importance, load_notification, load_smart_plug
    from src.data_loader.survey_loader.socioeconomics import load_socioeconomics
    print("Alle benötigten Module aus 'src' erfolgreich importiert.")
except ImportError as e:
    print(f"FEHLER beim Importieren von Modulen aus 'src': {e}")
    print("Bitte stelle sicher, dass der PROJECT_ROOT_NB korrekt ist und alle __init__.py Dateien in src und Unterordnern vorhanden sind.")
    raise e
except Exception as e:
    print(f"Ein unerwarteter Fehler beim Importieren ist aufgetreten: {e}")
    raise e

# --- Lade df_respondent_flexibility (Q9 & Q10 kombiniert) ---
print("\nLade df_respondent_flexibility (Q9 & Q10 kombiniert)...")
try:
    df_flex = create_respondent_flexibility_df() # Diese Funktion sollte jetzt ohne Argument funktionieren,
                                                 # da ihre internen Loader ihre Pfade selbst korrekt finden.
    print(f"df_flexibility geladen. Shape: {df_flex.shape}")
    if df_flex.empty:
        print("WARNUNG: df_flexibility ist leer! Die weitere Analyse wird nicht aussagekräftig sein.")
    else:
        if 'respondent_id' in df_flex.columns:
            df_flex['respondent_id'] = df_flex['respondent_id'].astype(str)
        print("Erste Zeilen von df_flexibility:")
        display(df_flex.head()) # display() ist besser für DataFrames in Jupyter
        print("\nInfo zu df_flexibility:")
        df_flex.info()
except FileNotFoundError as e:
    print(f"FEHLER beim Erstellen von df_flexibility (zugrundeliegende Q9/Q10 CSVs nicht gefunden?): {e}")
    df_flex = pd.DataFrame()
except Exception as e:
    print(f"Ein anderer Fehler beim Erstellen von df_flexibility: {e}")
    df_flex = pd.DataFrame()

# %% [markdown]
# ## Abschnitt 2: Laden und Integration weiterer relevanter Umfragedaten
# Hier laden wir die soziodemografischen Daten sowie Antworten zu anderen relevanten Fragen und führen sie mit `df_flex` zusammen, um unseren `master_df_f1` zu erstellen.

# %% [code]
master_df = pd.DataFrame()
if not df_flex.empty:
    master_df = df_flex.copy()

    data_loader_functions = {
        "demographics": (load_demographics, None), # Key in dict, loader_func, optional sub_key_in_dict_from_loader
        "socioeconomics": (load_socioeconomics, None),
        "attitudes": (load_attitudes, None),
        "q8_importance": (load_importance, "importance"), # load_importance gibt direkt DF zurück
        "q11_notification": (load_notification, "notification"),
        "q12_smartplug": (load_smart_plug, "smart_plug")
    }

    for group_key, (loader_func, specific_df_key) in data_loader_functions.items():
        print(f"\nLade und merge Daten für: {group_key}...")
        try:
            # Übergebe PROJECT_ROOT_NB an die Loader-Funktionen
            loaded_data = loader_func(PROJECT_ROOT_NB)
            
            if isinstance(loaded_data, dict): # Für Loader, die ein Dict von DFs zurückgeben (demographics, socioeconomics, attitudes)
                for sub_key, df_to_merge in loaded_data.items():
                    print(f"  Merging Untergruppe: {sub_key} (Shape: {df_to_merge.shape if not df_to_merge.empty else 'leer'})")
                    if not df_to_merge.empty and 'respondent_id' in df_to_merge.columns:
                        df_to_merge['respondent_id'] = df_to_merge['respondent_id'].astype(str)
                        master_df = pd.merge(master_df, df_to_merge, on="respondent_id", how="left", suffixes=('', f'_{sub_key}_dup'))
                    else:
                        print(f"    WARNUNG: DataFrame für '{sub_key}' aus '{group_key}' ist leer oder hat keine 'respondent_id'.")
            elif isinstance(loaded_data, pd.DataFrame): # Für Loader, die direkt einen DataFrame zurückgeben (Q8, Q11, Q12)
                df_to_merge = loaded_data
                print(f"  Merging: {group_key} (Shape: {df_to_merge.shape if not df_to_merge.empty else 'leer'})")
                if not df_to_merge.empty and 'respondent_id' in df_to_merge.columns:
                    df_to_merge['respondent_id'] = df_to_merge['respondent_id'].astype(str)
                    if group_key == "q8_importance": # Spezifische Behandlung für Q8 (wide format)
                        device_cols_q8 = [col for col in df_to_merge.columns if col != 'respondent_id']
                        if device_cols_q8:
                            df_q8_long = df_to_merge.melt(id_vars=['respondent_id'], value_vars=device_cols_q8,
                                                        var_name='device', value_name='importance_rating')
                            # Wichtig: 'device' Spaltennamen müssen exakt übereinstimmen mit master_df['device']
                            # Ggf. hier Mapping einfügen, wenn die Namen nicht direkt matchen
                            master_df = pd.merge(master_df, df_q8_long, on=["respondent_id", "device"], how="left")
                        else:
                             print(f"    WARNUNG: Keine Geräte-Spalten in '{group_key}' für melt-Operation gefunden.")
                    else: # Für Q11, Q12
                        master_df = pd.merge(master_df, df_to_merge, on="respondent_id", how="left", suffixes=('', f'_{group_key}_dup'))
                else:
                    print(f"    WARNUNG: DataFrame für '{group_key}' ist leer oder hat keine 'respondent_id'.")
            else:
                print(f"    WARNUNG: Unerwarteter Rückgabetyp von loader_func für '{group_key}'.")

        except FileNotFoundError as e:
            print(f"    FEHLER: Mindestens eine Datei für '{group_key}' nicht gefunden: {e}")
        except Exception as e:
            print(f"    Ein anderer Fehler beim Laden/Mergen von '{group_key}': {e}")
            import traceback
            traceback.print_exc()


    print("\n--- Master DataFrame (master_df_f1) nach Merging (Auszug) ---")
    if not master_df.empty:
        # Umbenennen des master_df für Klarheit im weiteren Verlauf des Notebooks
        master_df_f1 = master_df.copy() 
        del master_df # Aufräumen
        print("Erste 5 Zeilen von master_df_f1:")
        display(master_df_f1.head())
        print("\nInfo zu master_df_f1:")
        master_df_f1.info(verbose=True, show_counts=True)
        print(f"\nAnzahl unique Respondent IDs im master_df_f1: {master_df_f1['respondent_id'].nunique()}")
        print(f"Anzahl Zeilen im master_df_f1: {len(master_df_f1)}")
        
        # Überprüfe, ob die kritischen Spalten jetzt vorhanden sind
        print("\nÜberprüfung kritischer Spalten im master_df_f1:")
        critical_cols_for_f1 = ['device', 'max_duration_hours', 'incentive_choice', 'incentive_pct_required',
                                'age', 'gender', 'household_size', 'accommodation', 'electricity', # aus demographics
                                'q13_income', 'q14_education', 'q15_party', # aus socioeconomics
                                'importance_rating', # aus q8
                                'q11_notify', # aus q11
                                'q12_smartplug' # aus q12
                               ]
        for col in critical_cols_for_f1:
            if col in master_df_f1.columns:
                print(f"  Spalte '{col}': Vorhanden, Nicht-Null: {master_df_f1[col].notna().sum()}, Dtype: {master_df_f1[col].dtype}")
            else:
                print(f"  FEHLER: Spalte '{col}' fehlt im master_df_f1!")
    else:
        print("master_df ist leer. Überprüfe die Datenladeprozesse.")
        master_df_f1 = pd.DataFrame() # Sicherstellen, dass es existiert, auch wenn leer

# %% [markdown]
# ## Abschnitt 3: Deskriptive Analyse der Teilnahmebereitschaft (Q10 `incentive_choice`)
# Wie ist die generelle Bereitschaft zur Teilnahme an DR-Programmen?
# - Freiwillig (`yes_fixed`)
# - Nur mit Kompensation (`yes_conditional`)
# - Gar nicht (`no`)
#
# Analysiert gesamt, pro Gerät, und pro Dauer.

# %% [code]
if not master_df_f1.empty and 'incentive_choice' in master_df_f1.columns and 'device' in master_df_f1.columns:
    print("\n--- Verteilung der Teilnahmebereitschaft (incentive_choice) ---")
    
    # Gesamtverteilung
    print("\nGesamtverteilung der Teilnahmebereitschaft (Q10):")
    overall_participation_choice = master_df_f1['incentive_choice'].value_counts(normalize=True).mul(100).round(1)
    display(overall_participation_choice.to_frame(name="Anteil (%)"))
    fig_overall_choice = px.bar(overall_participation_choice.reset_index(), 
                                x='incentive_choice', y='proportion', # 'proportion' ist der neue Name von value_counts
                                title='Gesamte Teilnahmebereitschaft (Q10)', 
                                labels={'proportion':'Anteil (%)', 'incentive_choice': 'Teilnahme-Typ (Q10)'},
                                text_auto=True)
    fig_overall_choice.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
    fig_overall_choice.show()

    # Pro Gerät
    print("\nTeilnahmebereitschaft (Q10) pro Gerät:")
    participation_by_device = master_df_f1.groupby('device')['incentive_choice'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
    display(participation_by_device.pivot(index='device', columns='incentive_choice', values='percentage').fillna(0).round(1))
    fig_device_choice = px.bar(participation_by_device, x='device', y='percentage', color='incentive_choice',
                               barmode='group', title='Teilnahmebereitschaft pro Gerät (Q10)',
                               labels={'percentage':'Anteil (%)', 'device':'Gerät', 'incentive_choice':'Teilnahme-Typ (Q10)'},
                               text_auto=True)
    fig_device_choice.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
    fig_device_choice.update_xaxes(categoryorder='total descending') # Sortiere Geräte nach Gesamtanteil
    fig_device_choice.show()

    # Pro Dauer (max_duration_hours aus Q9)
    if 'max_duration_hours' in master_df_f1.columns:
        print("\nTeilnahmebereitschaft (Q10) pro angegebener max. Nichtnutzungsdauer (Q9):")
        master_df_f1['max_duration_hours_num'] = pd.to_numeric(master_df_f1['max_duration_hours'], errors='coerce')
        
        participation_by_duration = master_df_f1.dropna(subset=['max_duration_hours_num']).groupby('max_duration_hours_num')['incentive_choice'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
        display(participation_by_duration.pivot(index='max_duration_hours_num', columns='incentive_choice', values='percentage').fillna(0).round(1))
        fig_duration_choice = px.bar(participation_by_duration, x='max_duration_hours_num', y='percentage', color='incentive_choice',
                                     barmode='group', title='Teilnahmebereitschaft (Q10) nach max. Nichtnutzungsdauer (Q9)',
                                     labels={'percentage':'Anteil (%)', 'max_duration_hours_num':'Max. Nichtnutzungsdauer (Stunden, Q9)', 'incentive_choice':'Teilnahme-Typ (Q10)'},
                                     text_auto=True)
        fig_duration_choice.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
        fig_duration_choice.show()
else:
    print("master_df_f1 ist leer oder notwendige Spalten ('incentive_choice', 'device') fehlen für Abschnitt 3.")

# %% [markdown]
# ## Abschnitt 4: Detaillierte Analyse der monetären Kompensationsforderungen (Q10 `incentive_pct_required`)
# Für diejenigen, die eine Kompensation fordern (`incentive_choice == 'yes_conditional'`):
# - Wie hoch sind die Forderungen in Prozent?
# - Gibt es Unterschiede pro Gerät und pro Dauer?

# %% [code]
if not master_df_f1.empty and 'incentive_choice' in master_df_f1.columns and 'incentive_pct_required' in master_df_f1.columns:
    print("\n--- Analyse der monetären Kompensationsforderungen (incentive_pct_required) ---")
    df_compens_demand = master_df_f1[master_df_f1['incentive_choice'] == 'yes_conditional'].copy()
    
    df_compens_demand['incentive_pct_required_num'] = pd.to_numeric(df_compens_demand['incentive_pct_required'], errors='coerce')
    df_compens_demand.dropna(subset=['incentive_pct_required_num'], inplace=True)

    if not df_compens_demand.empty:
        print("\nDeskriptive Statistik für Kompensationsforderung (gesamt, für 'yes_conditional'):")
        display(df_compens_demand['incentive_pct_required_num'].describe().round(2).to_frame())

        fig_hist_total_compens = px.histogram(df_compens_demand, x='incentive_pct_required_num', nbins=20,
                                              title="Gesamtverteilung der geforderten Kompensation (%) für 'yes_conditional'",
                                              labels={'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
        fig_hist_total_compens.show()

        print("\n--- Kompensationsforderungen pro Gerät (für 'yes_conditional') ---")
        for device_name in sorted(df_compens_demand['device'].unique()):
            print(f"\nKompensationsforderung für: {device_name}")
            device_subset = df_compens_demand[df_compens_demand['device'] == device_name]
            if not device_subset.empty:
                display(device_subset['incentive_pct_required_num'].describe().round(2).to_frame())
                fig_hist_device = px.histogram(device_subset, x='incentive_pct_required_num', nbins=10,
                                               title=f"Verteilung Kompensation (%) für {device_name}",
                                               labels={'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
                fig_hist_device.show()

                if 'max_duration_hours_num' in device_subset.columns: # max_duration_hours_num wurde in Abschnitt 3 erstellt
                    fig_box_device_duration = px.box(device_subset.dropna(subset=['max_duration_hours_num']), 
                                                     x='max_duration_hours_num', y='incentive_pct_required_num',
                                                     title=f"Kompensation vs. Max. Dauer für {device_name}",
                                                     labels={'max_duration_hours_num': 'Max. Nichtnutzungsdauer (Stunden, Q9)', 
                                                             'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
                    fig_box_device_duration.show()
            else:
                print(f"Keine Daten für Kompensationsforderungen für {device_name} gefunden.")
    else:
        print("Keine Daten für 'yes_conditional' Teilnehmer mit validen Kompensationsforderungen gefunden.")
else:
    print("master_df_f1 ist leer oder notwendige Spalten fehlen für Abschnitt 4.")

# %% [markdown]
# ## Abschnitt 5: Einfluss ausgewählter soziodemografischer Faktoren auf Kompensationsforderungen
# Unterscheiden sich die Kompensationsforderungen (`incentive_pct_required_num` für `yes_conditional`) basierend auf z.B. Einkommen oder Alter?

# %% [code]
# Stelle sicher, dass df_compens_demand aus Abschnitt 4 existiert und nicht leer ist
if 'df_compens_demand' in locals() and not df_compens_demand.empty:
    print("\n--- Analyse des Einflusses soziodemografischer Faktoren auf Kompensationsforderungen ---")
    
    # Einkommen (q13_income)
    if 'q13_income' in df_compens_demand.columns:
        print("\nKompensationsforderung nach Einkommen:")
        income_categories_order = [
            "Unter 3.000 CHF", "3.000 - 5.000 CHF", "5.001 - 7.000 CHF",
            "7.001 - 10.000 CHF", "Über 10.000 CHF", "Keine Angabe"
        ]
        df_compens_demand['q13_income_cat'] = pd.Categorical(
            df_compens_demand['q13_income'], categories=income_categories_order, ordered=True
        )
        
        fig_income_compens = px.box(df_compens_demand.dropna(subset=['q13_income_cat', 'incentive_pct_required_num']),
                                     x='q13_income_cat', y='incentive_pct_required_num',
                                     title='Kompensationsforderung nach Haushaltseinkommen',
                                     labels={'q13_income_cat': 'Monatl. Haushaltsnettoeinkommen (Q13)',
                                             'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
        fig_income_compens.update_xaxes(tickangle=30)
        fig_income_compens.show()
        display(df_compens_demand.groupby('q13_income_cat', observed=False)['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2))
    else:
        print("Spalte 'q13_income' nicht im DataFrame für Kompensationsforderungen gefunden.")

    # Alter (age) - 'age' Spalte sollte numerisch sein aus Abschnitt 2
    if 'age' in df_compens_demand.columns:
        print("\nKompensationsforderung nach Alter:")
        # Altersgruppen bilden
        age_bins = [0, 25, 35, 45, 55, 65, 120] # Beispiel-Bins, anpassen!
        age_labels = ["<25", "25-35", "36-45", "46-55", "56-65", "65+"]
        df_compens_demand['age_group'] = pd.cut(df_compens_demand['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=True)
        
        fig_age_compens = px.box(df_compens_demand.dropna(subset=['age_group', 'incentive_pct_required_num']),
                                 x='age_group', y='incentive_pct_required_num',
                                 title='Kompensationsforderung nach Altersgruppe (Q1)',
                                 labels={'age_group': 'Altersgruppe',
                                         'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
        fig_age_compens.show()
        display(df_compens_demand.groupby('age_group', observed=False)['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2))
    else:
        print("Spalte 'age' nicht im DataFrame für Kompensationsforderungen gefunden.")
        
    # Hier weitere Analysen für Bildung (q14_education), Haushaltsgröße (household_size) etc. analog hinzufügen.
    # Beispiel für Bildung:
    if 'q14_education' in df_compens_demand.columns:
        print("\nKompensationsforderung nach Bildung:")
        # Bildung kategorien (anpassen an deine Daten!)
        # education_order = ["Grundschule", "Sekundarschule/Realschule", "Berufsausbildung/Lehre/Maturität", "Fachhochschule/Bachelor", "Universität/Master", "Promotion oder höher", "Keine Schulbildung", "Keine Angabe"]
        # df_compens_demand['q14_education_cat'] = pd.Categorical(df_compens_demand['q14_education'], categories=education_order, ordered=True)
        
        fig_edu_compens = px.box(df_compens_demand.dropna(subset=['q14_education', 'incentive_pct_required_num']),
                                     x='q14_education', y='incentive_pct_required_num',
                                     title='Kompensationsforderung nach höchstem Bildungsabschluss (Q14)',
                                     labels={'q14_education': 'Bildungsabschluss',
                                             'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
        fig_edu_compens.update_xaxes(tickangle=30)
        fig_edu_compens.show()
        display(df_compens_demand.groupby('q14_education')['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2))
    else:
        print("Spalte 'q14_education' nicht im DataFrame für Kompensationsforderungen gefunden.")
else:
    print("df_compens_demand ist leer oder nicht definiert, keine soziodemografische Analyse möglich.")

# %% [markdown]
# ## Abschnitt 6 (Optional): Einfluss von Geräte-Wichtigkeit (Q8) und DR-Affinität (Q11, Q12)

# %% [code]
if 'df_compens_demand' in locals() and not df_compens_demand.empty:
    print("\n--- Analyse zusätzlicher Einflussfaktoren (Optional) ---")

    # Q8 - Wichtigkeit des Geräts
    # 'importance_rating' sollte im master_df_f1 (und somit in df_compens_demand) sein,
    # wenn der Merge in Abschnitt 2 für Q8 (melted) erfolgreich war.
    if 'importance_rating' in df_compens_demand.columns:
        print("\nKompensationsforderung nach Wichtigkeit des Geräts (Q8):")
        # Stelle sicher, dass importance_rating numerisch ist
        df_compens_demand['importance_rating_num'] = pd.to_numeric(df_compens_demand['importance_rating'], errors='coerce')
        
        fig_importance = px.box(df_compens_demand.dropna(subset=['importance_rating_num', 'incentive_pct_required_num']),
                                 x='importance_rating_num', y='incentive_pct_required_num',
                                 title='Kompensationsforderung vs. Wichtigkeit des Geräts (Q8)',
                                 labels={'importance_rating_num': 'Wichtigkeit (1=sehr unwichtig, 5=sehr wichtig)',
                                         'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
        fig_importance.show()
        display(df_compens_demand.groupby('importance_rating_num')['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2))
    else:
        print("Spalte 'importance_rating' (aus Q8) nicht im DataFrame gefunden. Überprüfe den Merge-Schritt für Q8 in Abschnitt 2.")

    # Q11 - Benachrichtigungsbereitschaft & Q12 - Smart Plug Akzeptanz
    for col_q_akzeptanz, title_akzeptanz in [('q11_notify', 'Benachrichtigungsbereitschaft (Q11)'), 
                                            ('q12_smartplug', 'Smart Plug Akzeptanz (Q12)')]:
        if col_q_akzeptanz in df_compens_demand.columns:
            print(f"\nKompensationsforderung nach {title_akzeptanz}:")
            # Standardisiere Ja/Nein Antworten, falls nötig (z.B. "Ja ", "ja" -> "Ja")
            # Deine Preprocessing-Skripte sollten das schon erledigen (str.capitalize())
            # df_compens_demand[col_q_akzeptanz] = df_compens_demand[col_q_akzeptanz].str.strip().str.capitalize()
            
            fig_q_akzeptanz = px.box(df_compens_demand.dropna(subset=[col_q_akzeptanz, 'incentive_pct_required_num']),
                                     x=col_q_akzeptanz, y='incentive_pct_required_num',
                                     title=f'Kompensationsforderung vs. {title_akzeptanz}',
                                     labels={col_q_akzeptanz: title_akzeptanz.split('(')[0].strip(), # Kürzerer Labeltext
                                             'incentive_pct_required_num': 'Geforderte Kompensation (%)'})
            fig_q_akzeptanz.show()
            display(df_compens_demand.groupby(col_q_akzeptanz)['incentive_pct_required_num'].agg(['mean', 'median', 'std', 'count']).round(2))
        else:
            print(f"Spalte '{col_q_akzeptanz}' nicht im DataFrame gefunden.")
else:
    print("df_compens_demand ist leer oder nicht definiert, keine Analyse zusätzlicher Einflussfaktoren möglich.")

# %% [markdown]
# ## Abschnitt 7: Zusammenfassung der Kernergebnisse für F1 und Export
#
# *Hier fasst du die wichtigsten Erkenntnisse in Textform zusammen, basierend auf den obigen Analysen.*
# *Beispiel:*
# * "Die Analyse der Umfragedaten von XYZ Schweizer Haushalten zeigt, dass die Bereitschaft zur Teilnahme an Peak-Shaving-Programmen stark von den angebotenen Anreizen abhängt. Im Median fordern die teilnahmebereiten Haushalte eine Kompensation von 15% der Stromkosten für eine Verschiebung."
# * "Für das Gerät 'Waschmaschine' liegt die mittlere Kompensationsforderung bei A%, während für 'Geschirrspüler' B% gefordert werden. Dies deutet auf eine unterschiedliche Wertschätzung der ununterbrochenen Verfügbarkeit hin."
# * "Haushalte mit einem Nettoeinkommen über 10.000 CHF zeigen tendenziell [höhere/niedrigere/ähnliche] Kompensationsforderungen (Median C%) im Vergleich zu Haushalten mit einem Einkommen unter 3.000 CHF (Median D%)."
#
# *Code zum Speichern wichtiger Tabellen und Grafiken:*

# %% [code]
# Erstelle den outputs-Ordner, falls er nicht existiert
output_dir_f1 = PROJECT_ROOT_NB / "scripts" / "F1_Analyse_Kompensationsforderungen" / "outputs"
os.makedirs(output_dir_f1, exist_ok=True)
print(f"Ausgaben werden in '{output_dir_f1}' gespeichert.")

# Beispiel: Speichere deskriptive Statistiken der Kompensationsforderungen pro Gerät
if 'df_compens_demand' in locals() and not df_compens_demand.empty:
    try:
        stats_compens_per_device = df_compens_demand.groupby('device')['incentive_pct_required_num'].describe().round(2)
        stats_compens_per_device_path = output_dir_f1 / "f1_tabelle_kompensation_pro_geraet.csv"
        stats_compens_per_device.to_csv(stats_compens_per_device_path)
        print(f"Tabelle 'kompensation_pro_geraet.csv' gespeichert.")

        # Beispiel: Speichere einen der erstellten Plots (z.B. den Gesamt-Histogramm der Kompensation)
        # Dazu muss die Figur einer Variablen zugewiesen worden sein, z.B. fig_hist_total_compens
        if 'fig_hist_total_compens' in locals():
            fig_hist_total_compens.write_html(output_dir_f1 / "f1_plot_gesamte_kompensationsverteilung.html")
            # Für PNG: fig_hist_total_compens.write_image(output_dir_f1 / "f1_plot_gesamte_kompensationsverteilung.png") # Benötigt kaleido
            print(f"Plot 'gesamte_kompensationsverteilung.html' gespeichert.")
        
        # Du müsstest die anderen Plotly Express Figuren (px.) ebenfalls Variablen zuweisen, um sie zu speichern:
        # z.B. fig = px.bar(...) dann fig.write_html(...)
        
    except Exception as e:
        print(f"Fehler beim Speichern der Ergebnisse: {e}")
else:
    print("Keine Daten in df_compens_demand zum Speichern vorhanden.")

# %% [code]
print("\n--- F1 Analyse Notebook Ende ---")

Hinweis: __file__ nicht verfügbar, SCRIPT_DIR_F1 auf CWD gesetzt: /Users/jonathan/Documents/GitHub/PowerE/scripts/F1_Analyse_Kompensationsforderungen
Added project root '/Users/jonathan/Documents/GitHub/PowerE' to sys.path
Verwendeter Projekt-Root für dieses Notebook: /Users/jonathan/Documents/GitHub/PowerE
FEHLER beim Importieren von Modulen aus 'src': cannot import name 'load_smartplug' from 'src.data_loader.survey_loader.demand_response' (/Users/jonathan/Documents/GitHub/PowerE/src/data_loader/survey_loader/demand_response.py)
Bitte stelle sicher, dass der PROJECT_ROOT_NB korrekt ist und alle __init__.py Dateien in src und Unterordnern vorhanden sind.


ImportError: cannot import name 'load_smartplug' from 'src.data_loader.survey_loader.demand_response' (/Users/jonathan/Documents/GitHub/PowerE/src/data_loader/survey_loader/demand_response.py)